# // Getting NFL Data
___
The first step in getting this project underway is going to be getting **massive** amounts of NFL data from the web. I will be working in this notebook to "show my work" and for others to learn how to if they're curious. Ultimately, I'll also turn it into a regular `.py` Python script that you can run if you're so inclined.

For that, we're going to start out using the `requests` and `BeautifulSoup` libraries as well as API calls to glean information from:
- [Reddit recommendations](https://www.reddit.com/r/fantasyfootball/comments/34mbth/datasets_for_fantasy_football/)
- **nflgame** [here](http://wseaton.com/pulling-data-with-nflgame.html) and [here](https://pypi.org/project/nflgame/)
- [fantasydata](https://fantasydata.com/)
- [Pro Football Reference](https://www.pro-football-reference.com/)

In [2]:
# Importing our necessary libraries
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from bs4 import BeautifulSoup
%matplotlib inline

# Let's take a look at nflgame

___About___ `nflgame`:
nflgame is an API to retrieve and read NFL Game Center JSON data. It can work with real-time data, which can be used for fantasy football.

nflgame works by parsing the same JSON data that powers NFL.com’s live GameCenter. Therefore, nflgame can be used to report game statistics while a game is being played.

The package comes pre-loaded with game data from every pre- and regular season game from 2009 up until the present (Author tries to update it every week). Therefore, querying such data does not actually ping NFL.com.

However, if you try to search for data in a game that is being currently played, the JSON data will be downloaded from NFL.com at each request (so be careful not to inspect for data too many times while a game is being played). If you ask for data for a particular game that hasn’t been cached to disk but is no longer being played, it will be automatically cached to disk so that no further downloads are required.

- Going to work through a little example of one-off coding first.
- **Important:** If you haven't installed nflgame yet, you'll need to by running: `pip install nflgame`

In [2]:
import nflgame
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import seaborn as sns

%matplotlib inline

nflgame requires Python 2.6+ and does not yet work with Python 3
You are running Python version 3.6


SystemExit: 1

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


**Welp. Going to switch gears for a bit then...**

# Looking at fantasydata
- Require membership and paid subscription
- Going to avoid this for now since I want people to be able to follow along for free if they want.

## Scraping Pro Football Reference: